In [3]:
import csv
import sys
import datetime
import ast

In [39]:

csv.field_size_limit(sys.maxsize)

count = 0
output=[]

with open("/Users/kathyhuang/Projects/Personal/Design/Code/TextViz/coronavirus_project/news_api_nlp_2020-03-20.csv") as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    for row in reader:
        output.append(row['nlp_keywords'])
 
ast.literal_eval(output[0])

['warns',
 'invincible',
 'virus',
 'older',
 'health',
 'tedros',
 'coronavirus',
 'patrols',
 'media',
 'state',
 'york',
 'young',
 'closed']

In [14]:
import itertools

from lsh import cache, minhash # https://github.com/mattilyra/lsh

# a pure python shingling function that will be used in comparing
# LSH to true Jaccard similarities
def shingles(text, char_ngram=5):
    return set(text[head:head + char_ngram] for head in range(0, len(text) - char_ngram))


def jaccard(set_a, set_b):
    intersection = set_a & set_b
    union = set_a | set_b
    return len(intersection) / len(union)


def candidate_duplicates(document_feed, char_ngram=5, seeds=100, bands=5, hashbytes=4):
    char_ngram = 5
    sims = []
    hasher = minhash.MinHasher(seeds=seeds, char_ngram=char_ngram, hashbytes=hashbytes)
    if seeds % bands != 0:
        raise ValueError('Seeds has to be a multiple of bands. {} % {} != 0'.format(seeds, bands))
    
    lshcache = cache.Cache(num_bands=bands, hasher=hasher)
    for line in document_feed:
        line = line.decode('utf8')
        docid, headline_text = line.split('\t', 1)
        fingerprint = hasher.fingerprint(headline_text.encode('utf8'))
        
        # in addition to storing the fingerpring store the line
        # number and document ID to help analysis later on
        lshcache.add_fingerprint(fingerprint, doc_id=docid)

    candidate_pairs = set()
    for b in lshcache.bins:
        for bucket_id in b:
            if len(b[bucket_id]) > 1:
                pairs_ = set(itertools.combinations(b[bucket_id], r=2))
                candidate_pairs.update(pairs_)
    
    return candidate_pairs

In [18]:
hasher = minhash.MinHasher(seeds=100, char_ngram=5, hashbytes=4)
lshcache = cache.Cache(bands=10, hasher=hasher)
index = 0
documents = []
# read in the data file and add the first 100 documents to the LSH cache
with open("/Users/kathyhuang/Projects/Personal/Design/Code/TextViz/coronavirus_project/news_api_nlp_2020-03-20.csv") as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    for row in reader:
        documents.append(row['body'])
        lshcache.add_fingerprint(hasher.fingerprint(row['body']), index)
        index += 1

# for every bucket in the LSH cache get the candidate duplicates
candidate_pairs = set()
for b in lshcache.bins:
    for bucket_id in b:
        if len(b[bucket_id]) > 1: # if the bucket contains more than a single document
            pairs_ = set(itertools.combinations(b[bucket_id], r=2))
            candidate_pairs.update(pairs_)

In [16]:
candidate_pairs

{(6, 14), (12, 22), (13, 6), (13, 14), (91, 61), (93, 45)}

In [30]:
similarities = []
for ((line_a, docid_a), (line_b, docid_b)) in candidate_pairs:
    doc_a, doc_b = lines[line_a], lines[line_b]
    shingles_a = shingles(lines[line_a])
    shingles_b = shingles(lines[line_b])
    
    jaccard_sim = jaccard(shingles_a, shingles_b)
    fingerprint_a = set(hasher.fingerprint(doc_a.encode('utf8')))
    fingerprint_b = set(hasher.fingerprint(doc_b.encode('utf8')))
    minhash_sim = len(fingerprint_a & fingerprint_b) / len(fingerprint_a | fingerprint_b)
    similarities.append((docid_a, docid_b, jaccard_sim, minhash_sim))


"Estimating the proportion of people who've been infected with the coronavirus but don't have symptoms is a priority for researchers.Credit: Feature China/Barcroft Media/Getty\n\nAs coronavirus outbreaks surge worldwide, research teams are racing to understand a crucial epidemiological puzzle — what proportion of infected people have mild or no symptoms and might be passing the virus on to others. Some of the first detailed estimates of these covert cases suggest that they could represent some 60% of all infections.\n\nMany scientists have suspected that there is an undetected pool of covert cases showing limited to no symptoms, because an increasing number of infected people cannot be linked to known COVID-19 cases or travel to epidemic hotspots. Most people with mild infections would not be ill enough to seek medical help, and would probably slip past screening methods such as temperature checks, so the extent of the phenomenon and its role in virus transmission has remained elusive.

In [1]:
### try clustering
import os
import spacy

nlp = spacy.load('en_core_web_md')




In [6]:
directory = "/Users/kathyhuang/Projects/Personal/Design/Code/TextViz/coronavirus_project/news_files/"
#for filename in os.listdir(directory):
 #   with open(directory+filename) as csvfile:
 
output = []
with open("/Users/kathyhuang/Projects/Personal/Design/Code/TextViz/coronavirus_project/news_api_nlp_2020-03-20.csv") as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
    for row in reader:
        if len(row['body']) < 200:
            continue
        keyword_list = ast.literal_eval(row['nlp_keywords'])
        keywords = ' '.join(keyword_list)
        doc = nlp(keywords)
        output_dict = {}
        for index, word  in enumerate(keyword_list):
            output_dict[word] = doc[index].vector
        output.append(output_dict)

output[1] 

{'available': array([ 1.4507e-02, -1.1624e-01,  1.0342e-02, -1.4761e-01,  1.5760e-01,
        -1.4334e-01,  9.1898e-02, -2.4521e-01, -2.6863e-02,  1.5830e+00,
         2.3671e-02,  5.6386e-01,  1.3103e-02, -3.5812e-01,  2.9182e-01,
         8.6263e-02, -2.1640e-02,  2.3166e+00, -7.6147e-02, -5.3473e-01,
        -8.9396e-01, -1.5404e-01, -1.7019e-01,  1.9818e-02, -1.0969e-01,
         2.2048e-01, -8.5441e-02,  3.3733e-04,  1.6612e-01, -1.7404e-02,
         1.4849e-02, -5.7812e-01,  1.9454e-01, -4.0787e-03, -3.8443e-01,
         3.6314e-02, -1.7783e-01,  1.3600e-01,  2.1289e-01,  3.2930e-01,
        -2.5460e-01,  2.9294e-01, -1.2063e-01, -5.7370e-02, -2.6232e-01,
         4.5102e-01,  1.5652e-01, -1.2469e-01,  3.8356e-01, -1.3506e-01,
        -4.2586e-02, -6.3343e-01, -3.9293e-02,  3.0387e-01,  6.3688e-02,
        -2.6632e-01, -2.6717e-02, -5.5038e-01, -6.3944e-01, -1.2253e-01,
         1.6655e-01, -3.3855e-01,  8.0006e-02,  4.5584e-01,  3.8396e-01,
         3.7248e-01, -5.2798e-01,  3.6